In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExTraTrack/rcp_files/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
#            if int(Rows[i][84+l:88+l]) != 2071 and int(Rows[i][84+l:88+l]) != 2072:
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][9+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove -999.0 Data Points
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine DataFrames
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
    if Model == "Control":
        Year_Start = 1900
    elif Model == "RCP45":
        Year_Start = 2000
    elif Model == "RCP85":
        Year_Start = 2100
#        Year_Diff_A = Year_Start - 1985
#        Year_Diff_B = Year_Start - 1985 + 30
#        Year_Diff_C = Year_Start - 1985 + 60
    Origi_List_A = []
    Origi_List_B = []
    Origi_List_C = []
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Kinen = 728
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Kinen = 728
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Kinen = 728
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            if (DF_A["Orig Time(Z)"][k].year != 2071) and (DF_A["Orig Time(Z)"][k].year != 2072):
                List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            if (DF_B["Orig Time(Z)"][k].year != 2071) and (DF_B["Orig Time(Z)"][k].year != 2072):
                List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            if (DF_C["Orig Time(Z)"][k].year != 2071) and (DF_C["Orig Time(Z)"][k].year != 2072):
                List.append(DF_C[Vars[n]][k])
        Array[n] = List
#        Array[n] = numpy.concatenate((DF_A_Var[Vars[n]], DF_B_Var[Vars[n]], DF_C_Var[Vars[n]]))
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
# Ignore Orig Years of 2071-2072 Due to Data Corrupsion
    if Orig_Year < 2071 or Orig_Year >= 2073:
        if Orig_Year < 2071:
            Time_Year = Orig_Year + Year_Diff_A
        elif Orig_Year >= 2073:
            Time_Year = Orig_Year + Year_Diff_A - 2
        if Time_Year == Year:
            Count += 1
        else:
            Year += 1
            Count = 1
        if Count < 10:
            Code = "TC"+str(Year)+"0"+str(Count)
        else:
            Code = "TC"+str(Year)+str(Count)
        return (Code, Year, Count)

In [6]:
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
def Update_Year(Orig_Time, Year_Diff):
    if Orig_Time.year < 2071:
        Year_Update = Orig_Time.year + Year_Diff
    elif Orig_Time.year >= 2073:
        Year_Update = Orig_Time.year + Year_Diff - 2
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open File
def Create_ET_DF(File, Codes_DF, Year_Diff, ABC):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code_Orig = []
    Storm_Code_New = []
    Start_Time_Orig = []
    End_Time_Orig = []
    Start_Time_New = []
    End_Time_New = []
    Array_Orig = [[],[],[],[]]
    for i in range(len(Rows)):
        if int(Rows[i][33:37]) < 2071 or int(Rows[i][33:37]) >= 2073:
            Storm_Code_Orig.append(str(Rows[i][0:4]))
            Array_Orig[0].append(int(Rows[i][7:10]) * 6)
            Array_Orig[1].append(int(Rows[i][12]))
            Array_Orig[2].append(float(Rows[i][15:22]))
            Array_Orig[3].append(float(Rows[i][23:30]))
            Start_Time_Orig.append(datetime.datetime(year=int(Rows[i][33:37]), month=int(Rows[i][39:41]), \
            day=int(Rows[i][43:45]), hour=int(Rows[i][47:49])))
            End_Time_Orig.append(datetime.datetime(year=int(Rows[i][53:57]), month=int(Rows[i][59:61]), \
            day=int(Rows[i][63:65]), hour=int(Rows[i][67:69])))
    Array = numpy.array(Array_Orig)
#
# Use New Storm Codes and Times
    for i in range(len(Storm_Code_Orig)):
        for m in range(len(Codes_DF)):
            if Storm_Code_Orig[i] == Codes_DF["Orig Code"][m] and ABC == Codes_DF["ABC"][m]:
                Storm_Code_New.append(Codes_DF["New Code"][m])
    for i in range(len(Start_Time_Orig)):
        Start_Time_New.append(Update_Year(Start_Time_Orig[i], Year_Diff))
        End_Time_New.append(Update_Year(End_Time_Orig[i], Year_Diff))
#
# Create DataFrame to Store Data
    ET_DF = pandas.DataFrame({"Code": Storm_Code_New, "Path Type": Array[1], \
    "Start Time": Start_Time_New, "End Time": End_Time_New, "ET Duration (hr)": Array[0], \
    "Start SLP": Array[2], "End SLP": Array[3]})
    return (ET_DF)

In [9]:
# Combine ET DataFrames
def ET_Combine_DF(ET_DF_A, ET_DF_B, ET_DF_C, Main_DF):
# Concatenate Other Variables
    Vars = ["ET Duration (hr)", "Path Type", "Start SLP", "End SLP"]
    Total_Length = len(ET_DF_A) + len(ET_DF_B) + len(ET_DF_C)
    Array = numpy.zeros((4,Total_Length))
    for n in range(4):
        Array[n] = numpy.concatenate((ET_DF_A[Vars[n]], ET_DF_B[Vars[n]], ET_DF_C[Vars[n]]))
    Codes = list(ET_DF_A["Code"]) + list(ET_DF_B["Code"]) + list(ET_DF_C["Code"])
    Start_Time = list(ET_DF_A["Start Time"]) + list(ET_DF_B["Start Time"]) + list(ET_DF_C["Start Time"])
    End_Time = list(ET_DF_A["End Time"]) + list(ET_DF_B["End Time"]) + list(ET_DF_C["End Time"])
    ET_DF = pandas.DataFrame({"Code": Codes, "Path Type": Array[1], "Start Time": Start_Time, "End Time": End_Time, \
    "ET Duration (hr)": Array[0], "Start SLP": Array[2], "End SLP": Array[3]})
    ET_DF = ET_Algo_Fix(Main_DF, ET_DF)
    ET_DF = ET_Lat_Lon(Main_DF, ET_DF)
    ET_DF = ET_Distance(ET_DF)
    ET_DF = ET_Trop_Life(Main_DF, ET_DF)
    return (ET_DF)

In [10]:
# Edit False Alarms For ET Transition
def ET_Algo_Fix(Main_DF, ET_DF):
    Codes = ET_DF["Code"]
#    print (len(ET_DF))
    Start_Time_Fixes = []
    End_Time_Fixes = []
    for i in range(len(Codes)):
        Storm_DF = Main_DF[Main_DF["Code"] == Codes[i]]
        Storm_ET_DF = ET_DF[ET_DF["Code"] == Codes[i]]
        Start_Time = list(Storm_ET_DF["Start Time"])[0]
        End_Time = list(Storm_ET_DF["End Time"])[0]
        Final_Time = list(Storm_DF["Time(Z)"])[len(Storm_DF)-1]
#        print (Codes[i], Start_Time)
        After_Tropi = Storm_DF[(Storm_DF["Time(Z)"] >= Start_Time)]
        False_Alarm = After_Tropi[(After_Tropi["B"] <= 0) & (After_Tropi["VLT"] >= 100)].reset_index()
        if len(False_Alarm) > 0:
            Ref_Time = False_Alarm["Time(Z)"][len(False_Alarm)-1]
            New_After_Tropi = After_Tropi[After_Tropi["Time(Z)"] > Ref_Time].reset_index()
            Start_Time_Fix = 0
            End_Time_Fix = 0
            if len(New_After_Tropi) > 0:
                for j in range(len(New_After_Tropi)):
                    if New_After_Tropi["B"][j] > 15 or New_After_Tropi["VLT"][j] < 0:
                        if Start_Time_Fix == 0:
                            Start_Time_Fix = New_After_Tropi["Time(Z)"][j]
                    if New_After_Tropi["B"][j] > 15 and New_After_Tropi["VLT"][j] < 0:
                        if End_Time_Fix == 0:
                            End_Time_Fix = New_After_Tropi["Time(Z)"][j]
#                print (Codes[i], Start_Time_Fix, End_Time_Fix)
            if Start_Time_Fix == 0:
                Start_Time_Fix = Final_Time
                End_Time_Fix = Final_Time
            elif End_Time_Fix == 0:
                End_Time_Fix = Final_Time
            Start_Time_Fixes.append(Start_Time_Fix)
            End_Time_Fixes.append(End_Time_Fix)
        else:
            Start_Time_Fixes.append(Start_Time)
            End_Time_Fixes.append(End_Time)
#    print (len(Start_Time_Fixes))
    ET_DF["Start Time"] = Start_Time_Fixes
    ET_DF["End Time"] = End_Time_Fixes
    return (ET_DF)

In [11]:
# Add ET Transition Lat Lon Into ET DataFrame
def ET_Lat_Lon(Main_DF, ET_DF):
    Array = numpy.zeros((7, len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Array[0][i] = numpy.min(Storm["SLP(hPa)"])
        Array[1][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["SLP(hPa)"])
        Array[2][i] = float(Storm[Storm["Time(Z)"] == End_Time]["SLP(hPa)"])
        Array[3][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lon"])
        Array[4][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lat"])
        Array[5][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lon"])
        Array[6][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lat"])
    ET_DF["Min SLP"] = Array[0]
    ET_DF["Start SLP"] = Array[1]
    ET_DF["End SLP"] = Array[2]
    ET_DF["Start Lon"] = Array[3]
    ET_DF["Start Lat"] = Array[4]
    ET_DF["End Lon"] = Array[5]
    ET_DF["End Lat"] = Array[6]
    return (ET_DF)

In [12]:
# Calculate Distance Between ET Start and End Points
def ET_Distance(ET_DF):
    Array = numpy.zeros(len(ET_DF))
    for i in range(len(ET_DF)):
        Start_Lat = ET_DF["Start Lat"][i] * numpy.pi / 180
        End_Lat = ET_DF["End Lat"][i] * numpy.pi / 180
        Start_Lon = ET_DF["Start Lon"][i] * numpy.pi / 180
        End_Lon = ET_DF["End Lon"][i] * numpy.pi / 180
        Lat_Diff = End_Lat - Start_Lat
        Lon_Diff = End_Lon - Start_Lon
        Earth_Rad = 6378
        Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
        numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
        Array[i] = Distance
    ET_DF["ET Distance (km)"] = Array
    return (ET_DF)

In [13]:
# Calculate Duration Between Storm Formation and Start of ET Transition
def ET_Trop_Life(Main_DF, ET_DF):
    Array = numpy.zeros((2,len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Birth_Time = list(Storm["Time(Z)"])[0]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Trop_Time_Diff = Start_Time - Birth_Time
        Trop_Hours = int(Trop_Time_Diff.total_seconds()) / 3600
        Array[0][i] = Trop_Hours
        ET_Time_Diff = End_Time - Start_Time
        ET_Hours = int(ET_Time_Diff.total_seconds()) / 3600
        Array[1][i] = ET_Hours
    ET_DF["Tropical Duration (hr)"] = Array[0]
    ET_DF["ET Duration (hr)"] = Array[1]
    return (ET_DF)

In [14]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [15]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes):
    Storm_Name = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Name.append(Name)
    Codes["Name"] = Storm_Name
    return (Codes)

In [16]:
# Update Cyclone Code and Name DF to Only Storms That Completed ET
def Code_Update(Codes, ET_DF):
    Codes_ET = ET_DF["Code"]
    Codes_Final = Codes[Codes["New Code"].isin(Codes_ET)].reset_index()
    Codes_Final = Codes_Final.drop("index", axis=1)
    return (Codes_Final)

In [17]:
# Cyclone Type Based on ET Transition Algorithm
def ET_Update(Main_DF, ET_DF, Codes_DF):
    Codes = ET_DF["Code"]
    Final_DF = Main_DF[Main_DF["Code"].isin(Codes)].reset_index()
    ET_Type = []
    for i in range(len(Codes)):
        Storm_DF = Final_DF[Final_DF["Code"] == Codes[i]]
        Storm_ET_DF = ET_DF[ET_DF["Code"] == Codes[i]]
        Time = list(Storm_DF["Time(Z)"])
        B = list(Storm_DF["B"])
        VLT = list(Storm_DF["VLT"])
        Start_Time = list(Storm_ET_DF["Start Time"])[0]
        End_Time = list(Storm_ET_DF["End Time"])[0]
        for j in range(len(Time)):
            if Time[j] < Start_Time:
                if B[j] < 20 and VLT[j] > -50:
                    ET_Type.append("Tropical")
                else:
                    if B[j] > 20 and VLT[j] < -50:
                        ET_Type.append("Extratropical")
                    else:
                        ET_Type.append("Transition")
            elif Time[j] < End_Time:
                if B[j] > 10 or VLT[j] < 50:
                    if B[j] > 20 and VLT[j] < -50:
                        ET_Type.append("Extratropical")
                    else:
                        ET_Type.append("Transition")
                else:
                    ET_Type.append("Tropical")
            else:
                if B[j] > 10 and VLT[j] < 50:
                    ET_Type.append("Extratropical")
                else:
                    if B[j] < 10 and VLT[j] > 50:
                        ET_Type.append("Tropical")
                    else:
                        ET_Type.append("Transition")
    Final_DF["Type"] = ET_Type
    Names = []
    for i in range(len(Final_DF)):
        Code = Final_DF["Code"][i]
        Name = list(Codes_DF[Codes_DF["New Code"] == Code]["Name"])[0]
        Names.append(Name)
    Final_DF = Final_DF.drop("index", axis=1)
    Final_DF.insert(1, "Name", Names)
    ET_DF.insert(1, "Name", Codes_DF["Name"])
    return (Final_DF, ET_DF)

In [18]:
# Output File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output_V2.csv')
    Output_Diri = '/glade/u/home/whimkao//ExtraTrack/Output_Files/'
    DF.to_csv(Output_Diri+File_Name)

In [19]:
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_B_DF_Orig, Control_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.003_avg')
Control_C_DF_Orig, Control_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_DF, Control_Codes, Control_Year_Diffs = Combine_DF(Control_A_DF_Orig, Control_B_DF_Orig, Control_C_DF_Orig, \
Control_A_Storm_List_Orig, Control_B_Storm_List_Orig, Control_C_Storm_List_Orig, "Control")

In [20]:
Control_A_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.002.txt', Control_Codes, Control_Year_Diffs[0], "A")
Control_B_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.003.txt', Control_Codes, Control_Year_Diffs[1], "B")
Control_C_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.002.txt', Control_Codes, Control_Year_Diffs[2], "C")
Control_ET_DF = ET_Combine_DF(Control_A_ET_DF_Orig, Control_B_ET_DF_Orig, Control_C_ET_DF_Orig, Control_DF)

In [21]:
Name_List = Open_Name_List('Storm_Name_List.csv')
Control_Codes_Name = Assign_Name(Name_List, Control_Codes)
Control_Codes_Final = Code_Update(Control_Codes_Name, Control_ET_DF)
Control_DF_Final, Control_ET_Final = ET_Update(Control_DF, Control_ET_DF, Control_Codes_Final)

In [22]:
Output_File(Control_DF_Final, "Control", "Data")
Output_File(Control_ET_Final, "Control", "ET")
Output_File(Control_Codes_Final, "Control", "Codes")

In [23]:
Control_DF_Final[280:285]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Type
280,TC190105,Lydia,-59.59,28.39,921.57,64.1,-0.84,370.86,425.07,1901-09-22 18:00:00,Tropical
281,TC190105,Lydia,-60.54,28.77,920.25,64.4,-3.11,372.15,446.44,1901-09-23 00:00:00,Tropical
282,TC190105,Lydia,-60.94,28.96,919.96,65.9,-5.57,372.20,460.03,1901-09-23 06:00:00,Tropical
283,TC190105,Lydia,-61.51,29.14,917.78,65.1,-4.46,370.60,471.26,1901-09-23 12:00:00,Tropical
284,TC190105,Lydia,-61.38,29.60,916.02,67.2,-3.70,367.19,472.79,1901-09-23 18:00:00,Tropical


In [24]:
Control_ET_Final[4:7]

,Code,Name,Path Type,Start Time,End Time,ET Duration (hr),Start SLP,End SLP,Min SLP,Start Lon,Start Lat,End Lon,End Lat,ET Distance (km),Tropical Duration (hr)
4,TC190102,Harrison,1.0,1901-05-28 12:00:00,1901-05-30 18:00:00,54.0,991.56,1008.78,980.89,-43.71,30.04,-34.75,24.00,1112.101864,132.0
5,TC190103,Jessica,1.0,1901-07-12 00:00:00,1901-07-15 06:00:00,78.0,1006.39,1013.44,1001.93,-53.11,40.34,-30.00,37.75,2004.448676,78.0
6,TC190105,Lydia,1.0,1901-09-27 12:00:00,1901-09-29 18:00:00,54.0,953.76,970.00,913.33,-53.37,39.19,-42.82,48.53,1335.846025,384.0


In [25]:
Control_Codes_Final[4:7]

,ABC,Orig Code,New Code,Name
4,A,0078,TC190102,Harrison
5,A,0081,TC190103,Jessica
6,A,0089,TC190105,Lydia


In [26]:
RCP45_A_DF_Orig, RCP45_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45_avg')
RCP45_B_DF_Orig, RCP45_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg')
RCP45_C_DF_Orig, RCP45_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg')
RCP45_DF, RCP45_Codes, RCP45_Year_Diffs = Combine_DF(RCP45_A_DF_Orig, RCP45_B_DF_Orig, RCP45_C_DF_Orig, \
RCP45_A_Storm_List_Orig, RCP45_B_Storm_List_Orig, RCP45_C_Storm_List_Orig, "RCP45")

In [27]:
RCP45_A_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp45.txt', RCP45_Codes, RCP45_Year_Diffs[0], "A")
RCP45_B_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp45.002.txt', RCP45_Codes, RCP45_Year_Diffs[1], "B")
RCP45_C_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp45.003.txt', RCP45_Codes, RCP45_Year_Diffs[2], "C")
RCP45_ET_DF = ET_Combine_DF(RCP45_A_ET_DF_Orig, RCP45_B_ET_DF_Orig, RCP45_C_ET_DF_Orig, RCP45_DF)

In [28]:
Name_List = Open_Name_List('Storm_Name_List.csv')
RCP45_Codes_Name = Assign_Name(Name_List, RCP45_Codes)
RCP45_Codes_Final = Code_Update(RCP45_Codes_Name, RCP45_ET_DF)
RCP45_DF_Final, RCP45_ET_Final = ET_Update(RCP45_DF, RCP45_ET_DF, RCP45_Codes_Final)

In [29]:
Output_File(RCP45_DF_Final, "RCP45", "Data")
Output_File(RCP45_ET_Final, "RCP45", "ET")
Output_File(RCP45_Codes_Final, "RCP45", "Codes")

In [30]:
RCP45_DF_Final[830:835]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Type
830,TC200709,Hendrik,-85.13,22.62,994.96,27.1,-0.98,126.32,62.46,2007-09-13 00:00:00,Tropical
831,TC200709,Hendrik,-86.35,23.46,991.69,32.8,0.09,150.20,88.66,2007-09-13 06:00:00,Tropical
832,TC200709,Hendrik,-87.82,24.06,984.47,39.5,-0.08,179.97,136.56,2007-09-13 12:00:00,Tropical
833,TC200709,Hendrik,-88.65,24.53,972.49,46.5,-0.52,230.88,197.11,2007-09-13 18:00:00,Tropical
834,TC200709,Hendrik,-90.02,25.50,954.54,55.2,-2.12,274.98,268.57,2007-09-14 00:00:00,Tropical


In [31]:
RCP45_ET_Final[12:15]

,Code,Name,Path Type,Start Time,End Time,ET Duration (hr),Start SLP,End SLP,Min SLP,Start Lon,Start Lat,End Lon,End Lat,ET Distance (km),Tropical Duration (hr)
12,TC200404,Janet,1.0,2004-06-19 12:00:00,2004-06-22 06:00:00,66.0,1008.39,1009.81,1004.27,-72.84,35.82,-60.25,39.25,1172.230586,48.0
13,TC200407,Marco,2.0,2004-08-15 12:00:00,2004-08-15 18:00:00,6.0,1010.88,1006.79,998.88,-67.43,36.64,-64.99,37.94,260.036867,78.0
14,TC200409,Quinn,1.0,2004-09-09 12:00:00,2004-09-11 06:00:00,42.0,933.51,959.00,918.88,-60.82,37.35,-43.52,47.85,1823.677206,252.0


In [32]:
RCP45_Codes_Final[12:15]

,ABC,Orig Code,New Code,Name
12,A,0305,TC200404,Janet
13,A,0309,TC200407,Marco
14,A,0313,TC200409,Quinn


In [33]:
#RCP45_ET_Final[RCP45_ET_Final["Min SLP"] < 920]

In [34]:
RCP85_A_DF_Orig, RCP85_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85_avg')
RCP85_B_DF_Orig, RCP85_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.002_avg')
RCP85_C_DF_Orig, RCP85_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg')
RCP85_DF, RCP85_Codes, RCP85_Year_Diffs = Combine_DF(RCP85_A_DF_Orig, RCP85_B_DF_Orig, RCP85_C_DF_Orig, \
RCP85_A_Storm_List_Orig, RCP85_B_Storm_List_Orig, RCP85_C_Storm_List_Orig, "RCP85")

In [35]:
RCP85_A_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp85.txt', RCP85_Codes, RCP85_Year_Diffs[0], "A")
RCP85_B_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp85.002.txt', RCP85_Codes, RCP85_Year_Diffs[1], "B")
RCP85_C_ET_DF_Orig = Create_ET_DF(Diri+'etdetails_dtime900.rcp85.003.txt', RCP85_Codes, RCP85_Year_Diffs[2], "C")
RCP85_ET_DF = ET_Combine_DF(RCP85_A_ET_DF_Orig, RCP85_B_ET_DF_Orig, RCP85_C_ET_DF_Orig, RCP85_DF)

In [36]:
Name_List = Open_Name_List('Storm_Name_List.csv')
RCP85_Codes_Name = Assign_Name(Name_List, RCP85_Codes)
RCP85_Codes_Final = Code_Update(RCP85_Codes_Name, RCP85_ET_DF)
RCP85_DF_Final, RCP85_ET_Final = ET_Update(RCP85_DF, RCP85_ET_DF, RCP85_Codes_Final)

In [37]:
Output_File(RCP85_DF_Final, "RCP85", "Data")
Output_File(RCP85_ET_Final, "RCP85", "ET")
Output_File(RCP85_Codes_Final, "RCP85", "Codes")

In [38]:
RCP85_DF_Final[509:514]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Type
509,TC210703,Keith,-41.50,42.86,998.81,33.1,5.36,107.86,60.18,2107-10-10 18:00:00,Tropical
510,TC210703,Keith,-39.47,44.48,1001.43,28.4,12.82,31.97,-1.58,2107-10-11 00:00:00,Tropical
511,TC210703,Keith,-36.27,46.14,1000.82,29.7,20.95,-55.64,-65.01,2107-10-11 06:00:00,Extratropical
512,TC210703,Keith,-31.58,47.90,999.04,30.6,30.14,-136.59,-125.07,2107-10-11 12:00:00,Extratropical
513,TC210703,Keith,-26.00,49.75,995.64,34.4,37.77,-187.63,-180.42,2107-10-11 18:00:00,Extratropical


In [39]:
RCP85_ET_Final[10:13]

,Code,Name,Path Type,Start Time,End Time,ET Duration (hr),Start SLP,End SLP,Min SLP,Start Lon,Start Lat,End Lon,End Lat,ET Distance (km),Tropical Duration (hr)
10,TC210601,Dorcas,1.0,2106-09-25 00:00:00,2106-09-26 00:00:00,24.0,1008.73,1004.61,999.22,-75.85,34.95,-72.00,37.50,447.163505,96.0
11,TC210702,Janet,1.0,2107-04-12 00:00:00,2107-04-15 06:00:00,78.0,994.09,985.21,976.70,-65.33,26.15,-40.02,47.29,3198.473819,66.0
12,TC210703,Keith,3.0,2107-10-11 06:00:00,2107-10-11 06:00:00,0.0,1000.82,1000.82,946.47,-36.27,46.14,-36.27,46.14,0.000000,264.0


In [40]:
RCP85_Codes_Final[10:13]

,ABC,Orig Code,New Code,Name
10,A,0427,TC210601,Dorcas
11,A,0459,TC210702,Janet
12,A,0487,TC210703,Keith
